In [ ]:
import datetime

import numpy as np
import plotly.graph_objs as go
from dash import Dash, Input, Output, callback, dcc, html
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import functions as F
from shared.spark_config import create_spark_config

from libs.configuration import configure

env = configure(["../.env.development", "../.env.development.local"])
conf = create_spark_config("M4_Presentation")

In [ ]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
df_airports = (
    spark.table("dev.tier1.airports").select("icao", "name", "lat", "lon", "tz").toPandas()
)

In [ ]:
df_aircrafts = (
    spark.table("dev.tier1.aircrafts")
    .select("icao24", "registration", "doc8643_icao")
    .toPandas()
    .set_index("icao24")
)

In [ ]:
df_flight_signals = (
    spark.readStream.format("iceberg")
    .option("stream-from-timestamp", int(datetime.datetime(2023, 1, 18).timestamp()))
    .load("dev.tier1.flight_signals")
    .select(
        "icao24",
        "lat",
        "lon",
        "velocity",
        # "vertrate",
        "callsign",
        "heading",
        # "lastposupdate",
    )
)

In [ ]:
df_flights = None


def flight_signals_processor(df: DataFrame, batch_id: int):
    global df_flights

    df_flights = (
        df.toPandas().join(df_aircrafts, "icao24", "left")
    )

In [ ]:
stream = (
    df_flight_signals.writeStream.trigger(processingTime="15 seconds")
    .option(
        "checkpointLocation",
        f"abfss://warehouse@{env.DATASTORAGE_AZURE_ACCOUNTNAME}.dfs.core.windows.net/_checkpoints/M4_flight_signals",
    )
    .foreachBatch(flight_signals_processor)
    .start()
)

In [ ]:
bounding_box = ()
df_airports_bounded = None
df_flights_bounded = None


def update_bounded_data(new_bounding_box: tuple):
    """@param new_bounding_box = (lat_min, lat_max, lon_min, lon_max)"""
    global bounding_box, df_airports, df_flights, df_airports_bounded, df_flights_bounded

    if new_bounding_box != bounding_box:
        bounding_box = new_bounding_box
        lat_min, lat_max, lon_min, lon_max = bounding_box

        df_airports_bounded = df_airports[
            (df_airports.lat >= lat_min)
            & (df_airports.lat <= lat_max)
            & (df_airports.lon >= lon_min)
            & (df_airports.lon <= lon_max)
        ]

        df_flights_bounded = df_flights[
            (df_flights.lat >= lat_min)
            & (df_flights.lat <= lat_max)
            & (df_flights.lon >= lon_min)
            & (df_flights.lon <= lon_max)
        ]

In [ ]:
app = Dash("M4_Presentation")
app.layout = html.Div(
    html.Div(
        [
            html.Pre(id="log"),
            dcc.Graph(id="liveupdate-map"),
            dcc.Interval(id="interval-component", interval=20 * 1000, n_intervals=0),
        ]
    )
)


@callback(
    Output("liveupdate-map", "figure"),
    (Input("interval-component", "n_intervals"), Input("liveupdate-map", "relayoutData")),
)
def update_graph_live(n: int, layout_data: dict):
    global df_airports_bounded

    layout = go.Layout(
        margin={
            "t": 0,
            "r": 0,
            # "b": 20,
            "l": 0,
        },
        mapbox=dict(
            style="outdoors",
            zoom=1,
            uirevision=True,
            accesstoken="pk.eyJ1IjoiYmx1bmRlcmVyODQ0OCIsImEiOiJjbHJ4YnhucHQxNDJsMmxwY3R6NDg2c3IzIn0.CALLf90eS4adTscrk5MqEQ",
        ),
        height=800,
        legend=go.layout.Legend(orientation="h"),
    )

    figure = go.Figure(
        data=go.Scattermapbox(),
        layout=layout,
    )

    if layout_data is not None and "mapbox._derived" in layout_data:
        coords = layout_data["mapbox._derived"]["coordinates"]
        update_bounded_data((coords[2][1], coords[0][1], coords[0][0], coords[1][0]))

        figure = go.Figure(
            data=(
                go.Scattermapbox(
                    name="Airports",
                    mode="markers",
                    visible="legendonly",
                    lon=df_airports_bounded.lon,
                    lat=df_airports_bounded.lat,
                    customdata=np.stack(
                        (
                            df_airports_bounded.icao,
                            df_airports_bounded.name,
                            df_airports_bounded.tz,
                        ),
                        axis=-1,
                    ),
                    hovertemplate="<br>".join(
                        (
                            "icao=%{customdata[0]}",
                            "name=%{customdata[1]}",
                            "tz=%{customdata[2]}",
                        ),
                    ),
                    marker=go.scattermapbox.Marker(
                        size=4,
                        color="#930093",
                    ),
                ),
                go.Scattermapbox(
                    name="Flight Signals",
                    mode="markers",
                    lon=df_flights_bounded.lon,
                    lat=df_flights_bounded.lat,
                    customdata=np.stack(
                        (
                            df_flights_bounded.callsign,
                            df_flights_bounded.velocity,
                            df_flights_bounded.doc8643_icao,
                            df_flights_bounded.registration,
                            df_flights_bounded.icao24,
                        ),
                        axis=-1,
                    ),
                    hovertemplate="<br>".join(
                        [
                            "callsign=%{customdata[0]}",
                            "velocity=%{customdata[1]}",
                            "doc8643_icao=%{customdata[2]}",
                            "registration=%{customdata[3]}",
                            "icao24=%{customdata[4]}",
                        ]
                    ),
                    marker=go.scattermapbox.Marker(
                        angle=df_flights_bounded.heading,
                        allowoverlap=True,
                        symbol="airport",
                        size=10,
                        color="black",
                    ),
                ),
            ),
            layout=layout,
        )

    return figure


app.run(debug=True, jupyter_height=1000)